# Cleaning Data

### Import ✔️

In [1]:
import pathlib
import matplotlib

import os
import mne
import mne_bids
from mne.preprocessing import (ICA, create_eog_epochs, create_ecg_epochs, corrmap)

matplotlib.use('Qt5Agg')
mne.set_log_level('warning') #All mne functions will only spit out output if they have reason to.


### Check Epoch Quality

#### Input Data ✔️

In [2]:
epochs = mne.read_epochs(pathlib.Path('out_data')
            / 'epochs' / 'sub-01_ses-01_task-WM1isi0_epochs-epo.fif')
epochs

Number of events,503
Events,TsD-: 124WMD+: 127bgin: 124fix+: 128
Time range,-0.250 – 0.800 sec
Baseline,-0.250 – 0.000 sec


#### Reject Bad Epochs ❌

In [3]:
reject_criteria = dict(eeg=100e-6) # 100 µV, what should our rejection criteria be? 
#flat_criteria = dict(eeg=1e-6)           # 1 µV

epochs.drop_bad(reject=reject_criteria)
#epochs.drop_bad(reject=reject_criteria, flat=flat_criteria)

Number of events,260
Events,TsD-: 55WMD+: 75bgin: 57fix+: 73
Time range,-0.250 – 0.800 sec
Baseline,-0.250 – 0.000 sec


### ICA Analysis ❌

##### Read in new Raw Data

In [4]:
bids_root = pathlib.Path('out_data/bids_structure')

bids_path = mne_bids.BIDSPath(subject='01',
                              session='01',
                              task='WM1isi0',
                              datatype='eeg',
                              root=bids_root)

raw = mne_bids.read_raw_bids(bids_path) 
raw.load_data()

C:\Users\trevo\AppData\Local\Temp\ipykernel_8792\4141691380.py:9: RuntimeWarning: There are channels without locations (n/a) that are not marked as bad: ['SESS', 'CELL', 'bgin', 'fix+', 'WMD+', 'TsD-', 'TRSP', 'STI 014']
  raw = mne_bids.read_raw_bids(bids_path)
C:\Users\trevo\AppData\Local\Temp\ipykernel_8792\4141691380.py:9: RuntimeWarning: Not setting positions of 8 stim channels found in montage:
['SESS', 'CELL', 'bgin', 'fix+', 'WMD+', 'TsD-', 'TRSP', 'STI 014']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  raw = mne_bids.read_raw_bids(bids_path)


<RawBrainVision | sub-01_ses-01_task-WM1isi0_eeg.eeg, 137 x 361490 (361.5 s), ~378.0 MB, data loaded>

##### Specify EEG Data and EEG reference

In [5]:
raw_copy = raw.copy().pick_types(meg=False, eeg=True, eog=True)
raw_copy.set_eeg_reference(ref_channels=['E129'])
raw_copy.set_channel_types({'E8': 'eog'})
raw_copy.set_channel_types({'E25': 'eog'})
len(raw_copy.ch_names)
raw_copy

<RawBrainVision | sub-01_ses-01_task-WM1isi0_eeg.eeg, 129 x 361490 (361.5 s), ~356.0 MB, data loaded>

##### Apply a new filter

In [6]:
raw_copy.filter(l_freq=1, h_freq=None) 
#Change lowpass filter (high freq) later? Should be 40?

<RawBrainVision | sub-01_ses-01_task-WM1isi0_eeg.eeg, 129 x 361490 (361.5 s), ~356.0 MB, data loaded>

##### Template Matching

In [7]:
raws = list()
icas = list()

In [8]:
ica = ICA(n_components=15, max_iter='auto', random_state=97)
ica.fit(raw_copy)
ica

Method,fastica
Fit,28 iterations on raw data (361490 samples)
ICA components,15
Explained variance,100.0 %
Available PCA components,127
Channel types,eeg
ICA components marked for exclusion,—


In [9]:
raws.append(raw_copy)
icas.append(ica)

In [10]:
# use the first subject as template; use Fpz as proxy for EOG
raw = raws[0]
ica = icas[0]
eog_inds = ica.find_bads_eog(raw_copy, ch_name=['E8', 'E25'])
#eog_inds, eog_scores = ica.find_bads_eog(raw_copy, ch_name=['E8', 'E25'])

In [12]:
mne.preprocessing.corrmap(icas, template=(0, eog_inds[0]))

ValueError: No appropriate channels found for the given picks ([])

##### Draft 

##### Identify EOG Channels from EEG electrodes

#### EOG Artifact Detector

#### Bring Epochs data back in

In [ ]:
epochs = mne.read_epochs(pathlib.Path('out_data') / 'epochs' / 
                         'sub-01_ses-01_task-WM1isi0_epochs-epo.fif')
#epochs.selection is an array of all epochs that were not thrown out.
epochs_selection = epochs.selection
epochs_selection

In [ ]:
events, event_id = mne.events_from_annotations(raw)
events = events[epochs_selection]

In [ ]:
#Filtering to remove slow drifts
filt_raw = raw.copy().filter(l_freq=1., h_freq=None)
filt_raw

In [ ]:
tmin = epochs.tmin #-.250
tmax = epochs.tmax #.800

#Fit the epochs election into this epoch constructor
epochs_ica = mne.Epochs(raw,
                        events=events,
                        event_id=event_id,
                        tmin=tmin,
                        tmax=tmax,
                        preload=True)

In [ ]:
epochs_ica.info

In [ ]:
n_components = 0.999  

method = 'picard' 
max_iter = 1000  
fit_params = dict(fastica_it=5)
random_state = 42 

#Actually runs the ICA
ica = mne.preprocessing.ICA(n_components=n_components,
                            #max_pca_components=300,
                            method=method,
                            max_iter=max_iter,
                            fit_params=fit_params,
                            random_state=random_state)
ica.fit(epochs_ica)

In [ ]:
ica.plot_components(inst=epochs)

In [ ]:
epochs_cleaned = ica.apply(epochs.copy()) 

In [ ]:
#Ploting both after and before plots of epoched data
epochs_cleaned.plot(title='After ICA')
epochs.plot(title='Before ICA')